In [1]:
import tensorflow as tf
from Bio.PDB.PDBParser import PDBParser
from Bio.PDB.Atom import Atom
from Bio.PDB.Residue import Residue
from Bio.PDB.Chain import Chain
from Bio.PDB.Model import Model
from Bio.PDB.Structure import Structure
from Bio.PDB import PDBIO
import numpy as np
from ast import literal_eval

In [2]:
base_dir = "input_output/"
input_dir = "testing/"
pred_dir = "outputsTesting/"
NUM_DIMENSIONS = 3

In [56]:
AA_LETTERS = {
    'A': 'ALA',
    'R': 'ARG',
    'N': 'ASN',
    'D': 'ASP',
    'C': 'CYS',
    'E': 'GLU',
    'Q': 'GLN',
    'G': 'GLY',
    'H': 'HIS',
    'I': 'ILE',
    'L': 'LEU',
    'K': 'LYS',
    'M': 'MET',
    'F': 'PHE',
    'P': 'PRO',
    'S': 'SER',
    'T': 'THR',
    'W': 'TRP',
    'Y': 'TYR',
    'V': 'VAL'
}

In [45]:
class Protein:
    """Stores amino acid sequence and structure.

    Attributes:
        id_: name of protein
        primary: string of amino acid sequence
        actual_tertiary: np array
        mask: np array of 1 and 0s
        pred_tertiary: np array
    """

    _aa_dict = {
        'A': '0',
        'C': '1',
        'D': '2',
        'E': '3',
        'F': '4',
        'G': '5',
        'H': '6',
        'I': '7',
        'K': '8',
        'L': '9',
        'M': '10',
        'N': '11',
        'P': '12',
        'Q': '13',
        'R': '14',
        'S': '15',
        'T': '16',
        'V': '17',
        'W': '18',
        'Y': '19'
    }
    _mask_dict = {'-': '0', '+': '1'}
    _aa_dict = dict((v, k) for k, v in _aa_dict.items())

    def __init__(self, id_, primary, actual_tertiary, mask, pred_tertiary):
        self.id_ = id_
        self.primary = primary
        self.actual_tertiary = actual_tertiary
        self.pred_tertiary = pred_tertiary
        self.mask = mask
        self.int_to_aa()

    def int_to_aa(self):
        integers = list(self.primary.astype('str'))
        aa = "".join([self._aa_dict[integer] for integer in integers])
        self.primary = aa

In [3]:
def read_tertiary_file(path):
    """Read .tertiary file"""

    coords = np.transpose(np.loadtxt(path))

    return coords

In [5]:
np.fliplr(read_tertiary_file(base_dir + pred_dir + 'T0859.tertiary'))

array([[    0.        ,     0.        ,     0.        ],
       [  -99.12168121,    74.59438324,    76.60613251],
       [ -240.68289185,    32.3781929 ,    39.43859863],
       ...,
       [-1055.63903809, 14394.14550781, 13731.828125  ],
       [ -983.13934326, 14515.31933594, 13768.140625  ],
       [ -866.41516113, 14540.1640625 , 13673.4765625 ]])

In [59]:
def read_and_decode(filename_queue):
    """Parse a single instance of a tf record.
    
    Args:
        filename_queue: tf queue
        
    Returns:
        id_:tf tensor
        primary: tf tensor
        tertiary: tf tensor
        mask: tf tensor
    """

    reader = tf.TFRecordReader()
    _, serialized_example = reader.read(filename_queue)

    context, features = tf.parse_single_sequence_example(
        serialized_example,
        context_features={'id': tf.FixedLenFeature((1, ), tf.string)},
        sequence_features={
            'primary': tf.FixedLenSequenceFeature((1, ), tf.int64),
            'tertiary': tf.FixedLenSequenceFeature(
                (NUM_DIMENSIONS, ),
               tf.float32,
               allow_missing=True),
            'mask':tf.FixedLenSequenceFeature(
                (1, ), 
                tf.float32, 
                allow_missing=True)
        })

    id_ = context['id'][0]
    primary = tf.to_int32(features['primary'][:, 0])
    tertiary = features['tertiary']
    mask = features['mask'][:, 0]
    return id_, primary, tertiary, mask

In [48]:
def tf_record_to_dict(tf_path, tertiary_dir):
    """Convert tfrecord to a list of Proteins.
    
    Args:
        tf_path: path to tf record
        tertiary_dir: directory that holds .tertiary files
        
    Returns:
        list of Proteins
    """

    tf.reset_default_graph()
    with tf.Session() as sess:
        proteins = []
        init_op = tf.group(
            tf.global_variables_initializer(),
            tf.local_variables_initializer())
        sess.run(init_op)

        filename_queue = tf.train.string_input_producer(
            [tf_path],
            shuffle=False)

        attributes = read_and_decode(filename_queue)
        coord = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(coord=coord)

        size = sum(1 for _ in tf.python_io.tf_record_iterator(tf_path))
        for i in range(size):

            id_, primary, tertiary, mask = sess.run(list(attributes))
            id_ = id_.decode("utf-8")
            try:
                pred_coords = read_tertiary_file(
                    tertiary_dir + id_ + '.tertiary')
            except (FileNotFoundError, OSError):
                pred_coords = np.array([])
            protein = Protein(id_, primary, tertiary, mask, pred_coords)
            proteins.append(protein)
        coord.request_stop()
        coord.join(threads)

        return proteins

In [49]:
proteins = tf_record_to_dict(base_dir + input_dir + '1', base_dir + pred_dir)

In [57]:
def create_pdb_file(protein, save_dir):
    """Create a PDB file from a Protein and return the structure.
    
    Args:
        protein: Protein
        save_dir: directory to save pdb files
        
    Returns:
        None
    """

    def create_structure(coords, pdb_type, remove_masked):
        """Create the structure.
        
        Args:
            coords: 3D coordinates of structure
            pdb_type: predict or actual structure
            remove_masked: whether to include masked atoms. If false, the masked atoms 
                  have coordinates of [0,0,0].
                  
        Returns:
            structure
        """
        
        name = protein.id_
        structure = Structure(name)
        model = Model(0)
        chain = Chain('A')
        for i, residue in enumerate(protein.primary):
            residue = AA_LETTERS[residue]
            if int(protein.mask[i]) == 1 or remove_masked == False:
                new_residue = Residue((' ', i + 1, ' '), residue, '    ')
                j = 3 * i
                atom_list = ['N', 'CA', 'CB']
                for k, atom in enumerate(atom_list):
                    new_atom = Atom(
                        name=atom,
                        coord=coords[j + k, :],
                        bfactor=0,
                        occupancy=0,
                        altloc=' ',
                        fullname=" {} ".format(atom),
                        serial_number=0)
                    new_residue.add(new_atom)
                chain.add(new_residue)
        model.add(chain)
        structure.add(model)
        io = PDBIO()
        io.set_structure(structure)
        io.save(save_dir + name + '_' + pdb_type + '.pdb')
        return structure

    coords = np.around(protein.actual_tertiary, 1)
    structure = create_structure(coords, "actual", remove_masked=True)

    if protein.pred_tertiary.size != 0:
        coords = np.around(protein.pred_tertiary, 1)
        structure = create_structure(coords, "pred", remove_masked=False)


def create_pdb_files(proteins, save_dir):
    """Create multiple pdb files
    
    Args:
        proteins: list of Proteins
        save_dir: directory to save pdb files
    
    Returns:
        None
    """

    for protein in proteins:
        create_pdb_file(protein, save_dir)

In [58]:
create_pdb_files(proteins, base_dir + "d")

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/Bio/PDB/Atom.py:189: PDBConstructionWarning: Used element 'N' for Atom (name=N) with given element None
  warnings.warn(msg, PDBConstructionWarning)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/Bio/PDB/Atom.py:189: PDBConstructionWarning: Used element 'C' for Atom (name=CA) with given element None
  warnings.warn(msg, PDBConstructionWarning)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/Bio/PDB/Atom.py:189: PDBConstructionWarning: Used element 'C' for Atom (name=CB) with given element None
  warnings.warn(msg, PDBConstructionWarning)


In [ ]:
io = PDBIO()
io.set_structure(structure)
io.save(base_dir + "test_out.pdb")

In [ ]:
parser = PDBParser(PERMISSIVE=1)
structure_id = "3srp"
# filename = base_dir  + "3srp.pdb"
filename = base_dir  + "test_out.pdb"
structure = parser.get_structure(structure_id, filename)

In [ ]:
model = structure[0]
chain = model['A']
residue = chain[25]
atom = residue['N']
print(structure)
print(model)
print(chain)
print("\n\n\n")

print(residue)
print(residue.id)
print(residue.resname)
print(residue.segid)
print(list(residue.get_atoms()))
print(atom)

In [ ]:
atom.coord

In [ ]:
test_out = np.transpose(np.array(all_numbers))#.shape

In [ ]:
test_coords

In [63]:
np.around(5.34534, 1)

5.3